<a href="https://colab.research.google.com/github/greymattar/ECG_WGAN_AugmentationClassification/blob/main/ML_models_GenTest_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount ('/content/gdrive')
import sys
import os
#sys.path.append('/content/gdrive/MyDrive/Augmentation-of-ECG-Training-Dataset-with-CGAN-main')
os.chdir('/content/gdrive/MyDrive/Augmentation-of-ECG-Training-Dataset-with-CGAN-main')
print(os.getcwd())

Mounted at /content/gdrive
/content/gdrive/MyDrive/Augmentation-of-ECG-Training-Dataset-with-CGAN-main


In [ ]:
import pandas as pd

myPath_base = os.path.join("PycharmProjects\\paper2_gen_data\\", "MIT_BIH", "multiclass", "01_cond\\screened","gb_L_cond_screened.json")
df =pd.read_json(myPath_base)
print(df.head())

        0         1         2         3         4         5         6    \
0 -0.596866 -0.572873 -0.558806 -0.579448 -0.579438 -0.526910 -0.563536   
1 -0.713581 -0.718979 -0.690804 -0.733377 -0.730963 -0.695825 -0.726390   
2 -0.237402  0.010355 -0.027357  0.109423  0.031767  0.177342  0.141292   
3 -0.700903 -0.709553 -0.674504 -0.719653 -0.720079 -0.684136 -0.719242   
4 -0.677850 -0.699551 -0.647607 -0.694001 -0.697317 -0.658262 -0.709017   

        7         8         9    ...       246       247       248       249  \
0 -0.518285 -0.518867 -0.498051  ... -0.402239 -0.387153 -0.387750 -0.432009   
1 -0.687446 -0.695721 -0.681294  ... -0.335653 -0.322302 -0.344544 -0.414038   
2  0.192209  0.162326  0.182625  ... -0.466923 -0.553367 -0.527034 -0.506896   
3 -0.681613 -0.694466 -0.678787  ... -0.330393 -0.320869 -0.338802 -0.412933   
4 -0.677262 -0.704213 -0.680731  ... -0.330031 -0.339077 -0.343634 -0.425239   

        250       251       252       253       254       255  
0 -0

In [ ]:
import json
import os

classes2keep = ['N','/', 'A', 'L', 'R', 'f', 'j','F','a']
classes2keep_folder = ['N','P', 'A', 'L','R', 'f', 'j','F','a']

data = []  # List to store the beat values and class labels

for cl in classes2keep_folder:

    filename = "gb_{}_cond_screened.json".format(cl)
    file_path = os.path.join("PycharmProjects\\paper2_gen_data\\", "MIT_BIH", "multiclass", "01_cond\\screened", filename)  # Replace with the actual path to the JSON files

    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            json_data = json.load(file)

            # Extract beat values and class label
            for row in json_data:
                beat_values = row
                class_label = cl

                data.append([beat_values, class_label])

# Print the joined data

print(data[1][1])


N


In [ ]:
print(data[15000][1])

P


In [ ]:
import json

# Save data as JSON
output_file = 'gb_together.json'  # Replace with the desired path and filename for the output JSON file

with open(output_file, 'w') as file:
    json.dump(data, file)

print("Data saved successfully to", output_file)


Data saved successfully to gb_together.json


In [ ]:
pip install wfdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 16.8 MB/s eta 0:00:00


In [ ]:
import copy
import csv
import datetime
import json
import os
from collections import Counter

import matplotlib.pyplot as plt
import numpy as np
import torch

import torch.optim as optim

import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset

import sys
#sys.path.append('/content/gdrive/MyDrive/Augmentation-of-ECG-Training-Dataset-with-CGAN-main')

from ekg_class import dicts
from model_ac_wgan_gp_ecg import Disc_ac_wgan_gp_1d, Gen_ac_wgan_gp_1d, initialize_weights_1d
from utils import gradient_penalty, normalize, grid_plot_save, grid_plot

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

num2descr, letter2num, letter2descr, num2letter = dicts()
start_time = datetime.datetime.now()
print(("\n" + "*" * 50 + "\n\t\tstart time:      {0:02d}:{1:02d}:{2:02.0f}\n" + "*" * 50).format(
    start_time.hour, start_time.minute, start_time.second))

#drive = "F:"
drive = ""
myPath_base = os.path.join(drive, "")
myPath_dataset = os.path.join(myPath_base, "")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dry_run = False
if dry_run:
    NUM_EPOCHS = 1
else:
    NUM_EPOCHS = 12
BATCH_SIZE = 16
CHANNELS_IMG = 1
CRITIC_ITERATIONS = 5
FEATURES_DISC = 64
FEATURES_GEN = 64
GEN_EMBEDDING = 100
LAMBDA_GP = 10
LEARNING_RATE = 1e-4
Z_DIM = 100

tr_ts_ratio = 0.9     # "training set / test set" split ratio
len_ratio = 1     # to study the effect of support set number of samples (shorter train sets)
if '.' in str(len_ratio):
    len_ratio_str = str(len_ratio).replace('.', "")
else:
    len_ratio_str = str(len_ratio)

# transforms = transforms.Compose(
#    [
#        transforms.Resize(IMG_SIZE),
#        transforms.ToTensor(),
#        transforms.Normalize(
#            [0.5 for _ in range(CHANNELS_IMG)], [0.5 for _ in range(CHANNELS_IMG)]),
#    ]
# )


with open(os.path.join(myPath_dataset, "record_X_y_adapt_win_bef075_aft075_Normalized.json"), "r") as f:
    data = json.load(f)

stats_all_classes = Counter(np.asarray(data, dtype=object)[:, 2])

all_classes = ['/', 'A', 'E', 'F', 'J', 'L', 'N', 'Q', 'R', 'S', 'V', 'a', 'e', 'f', 'j']
all_classes_folder = ['P', 'A', 'E', 'F', 'J', 'L', 'N', 'Q', 'R', 'S', 'V', 'a', 'e', 'f', 'j']

all_classes.remove('Q')
all_classes.remove('S')
# all_classes.remove('V')      # why remove this? plots show no-uniform pattern

# %%%%%%%%%%%%%%%%%    begin classes statistics      %%%%%%%%%%%%%%%%%%
"""
stats_all_classes =
        {
        'N': 75052,   'L': 8075,  'R': 7259,  'V': 7130,
        '/': 7028,    'A': 2546,  'f': 982,   'F': 803,
        'j': 229,     'a': 150,   'E': 106,   'J': 83,
        'Q': 33,      'e': 16,    'S': 2
        }
P: Paced beat
A: Atrial Premature contraction
E: Ventricular Escape beat
F: Fusion of Ventricular and Normal beat
J: Nodal (junctional) Premature Beat
L: Left bundle branch block beat
N: Normal beat
Q: Unclassifiable beat
R: Right bundle branch block beat
S: Premature or ectopic supraventricular beat
V: Premature Ventricular Contraction
a: Aberrated Atrial Premature beat
e: Atrial escape beat
f: Fusion of paced and normal beat
j: Nodal (junctional) escape beat
"""
# %%%%%%%%%%%%%%%%%    end classes statistics    %%%%%%%%%%%%%%%%%%%

# %%%%%%%%%%%%%%%%       begin MIT-BIH Dataset      %%%%%%%%%%%%%%%%%%%%%%%%%%%%

d_set = "MIT_BIH"
# Note: '/' and 'P' are the same
#classes2keep = ['/', 'A', 'L', 'N', 'R', 'f', 'j']
#classes2keep_folder = ['P', 'A', 'L', 'N', 'R', 'f', 'j']
classes2keep = ['N','/', 'A', 'L', 'R', 'f', 'j','F','a']
classes2keep_folder = ['N','P', 'A', 'L','R', 'f', 'j','F','a']

# d_set = "ECG5000\\"
# classes2keep = [1, 2, 3, 4, 5]

myPath_save = os.path.join(myPath_base, "PycharmProjects\\paper2_gen_data\\", d_set, "multiclass",
                            "genbeats_ac_wgan_gp_cl_{}_len_ratio_{}".format(classes2keep_folder, len_ratio_str))
os.makedirs(myPath_save, exist_ok=True)

NUM_CLASSES = len(classes2keep)
IMG_SIZE = 256

#with open(os.path.join(myPath_dataset, "record_X_y_adapt_win_bef075_aft075_Normalized.json"), "r") as f:
    #data = json.load(f)

stats_all_classes = Counter(np.asarray(data, dtype=object)[:, 2])

# create dictionary of data to be kept
vals = []
for idx in range(len(classes2keep)):
    vals.append([])

data2keep_dict = dict(zip(classes2keep, vals))
data_train_dict = copy.deepcopy(data2keep_dict)
data_test_dict = copy.deepcopy(data2keep_dict)

for item in data:
    if item[2] in classes2keep:
        data2keep_dict[item[2]].append(item[1])
del data

if '/' in classes2keep:
    temp = data2keep_dict['/']
    data2keep_dict.pop('/')
    data2keep_dict['P'] = temp

# randomly splitting the dataset into train and test sets
for key in classes2keep_folder:
    val_len = len(data2keep_dict[key])
    idx_train = torch.randperm(val_len)[:int(tr_ts_ratio * val_len)]
    idx_test = torch.randperm(val_len)[int(tr_ts_ratio * val_len)+1:]
    data_train_dict[key] = [data2keep_dict[key][idx] for idx in idx_train]
    data_test_dict[key] = [data2keep_dict[key][idx] for idx in idx_test]

X_train = []
y_train = []
# data_train_dict contains all the training data
# X_train contains only a portion of training data (len_ratio), to study of size of training set on
#           classification metrics

a = 0
for key in classes2keep_folder:
    temp = data_train_dict[key]
    length = int(len_ratio*len(temp))
    X_train.extend(temp[:length])
    idx = [classes2keep_folder.index(key)] * length
    # print('key {}, idx {}'.format(key, idx))
    y_train.extend(idx)

X_test = []
y_test = []
for key in classes2keep_folder:
    X_test.extend(data_test_dict[key][:])
    idx = [classes2keep_folder.index(key)] * len(data_test_dict[key])
    y_test.extend(idx)

y_test_stat = Counter(y_test)
y_train_stat = Counter(y_train)

print("train: {}".format(y_train_stat))
print("test: {}".format(y_test_stat))


**************************************************
		start time:      22:15:11
**************************************************
train: Counter({0: 67546, 3: 7267, 4: 6533, 1: 6325, 2: 2291, 5: 883, 7: 722, 6: 206, 8: 135})
test: Counter({0: 7505, 3: 807, 4: 725, 1: 702, 2: 254, 5: 98, 7: 80, 6: 22, 8: 14})


In [ ]:
pip install pywt scikit-learn


ERROR: Could not find a version that satisfies the requirement pywt (from versions: none)
ERROR: No matching distribution found for pywt


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import svm, datasets
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import pywt
from sklearn import svm


In [ ]:
# Initialize feature vector and labels
feature_vector = []
labels = []

# Iterate over the beats in the dictionary
for cls, beats in data_train_dict.items():
    for beat in beats:
        # Perform wavelet decomposition at 4 levels
        coeffs = pywt.wavedec(beat, 'db6', level=4)

        # Extract approximation coefficients at level 4
        approx_coeffs = coeffs[0]

        # Append approximation coefficients to feature vector
        feature_vector.append(approx_coeffs)

        # Append label for the beat
        labels.append(cls)  # Assuming the record number is the label

# Convert feature vector and labels to numpy arrays
feature_vector = np.array(feature_vector)
labels = np.array(labels)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(feature_vector, labels, train_size=0.8, random_state = 0)

In [ ]:
linear = svm.SVC(kernel='linear', C=1, decision_function_shape='ovo').fit(X_train, y_train)
rbf = svm.SVC(kernel='rbf', gamma=1, C=1, decision_function_shape='ovo').fit(X_train, y_train)
poly = svm.SVC(kernel='poly', degree=3, C=1, decision_function_shape='ovo').fit(X_train, y_train)
sig = svm.SVC(kernel='sigmoid', C=1, decision_function_shape='ovo').fit(X_train, y_train)

In [ ]:
#linear_pred = linear.predict(X_test)
poly_pred = poly.predict(X_test)
#rbf_pred = rbf.predict(X_test)
#sig_pred = sig.predict(X_test)

In [ ]:
#accuracy_lin = linear.score(X_test, y_test)
accuracy_poly = poly.score(X_test, y_test)
#accuracy_rbf = rbf.score(X_test, y_test)
#accuracy_sig = sig.score(X_test, y_test)
#print("Accuracy Linear Kernel:", accuracy_lin)
print("Accuracy Polynomial Kernel:", accuracy_poly)
#print("Accuracy Radial Basis Kernel:", accuracy_rbf)
#print("Accuracy Sigmoid Kernel:", accuracy_sig)

Accuracy Polynomial Kernel: 0.9328691110869328


In [ ]:
# creating a confusion matrix
cm_lin = confusion_matrix(y_test, linear_pred)
cm_poly = confusion_matrix(y_test, poly_pred)
cm_rbf = confusion_matrix(y_test, rbf_pred)
cm_sig = confusion_matrix(y_test, sig_pred)
print(cm_lin)
print(cm_poly)
print(cm_rbf)
print(cm_sig)

[[  193     0    10   260    13    20     0     0     0]
 [    1    43     1    87     1     0     0     0     0]
 [    2     0   749   689     7     7     0     0     0]
 [   29    18   110 13226    66    47     1     2     0]
 [   20     0    11   159  1094     3     0     0     0]
 [    8     0     0   630     0   660     0     0     0]
 [    4     0     4    12     0     1     4     0     0]
 [    0     0     8   148     5     0     0     0     0]
 [    0     0     1    28     0     0     0     0     0]]
[[  241     0    13   239     0     2     1     0     0]
 [    0    93     1    37     1     1     0     0     0]
 [    6     0  1330   111     3     3     0     1     0]
 [   21    11    62 13334    33    36     0     1     1]
 [    0     0     5    51  1230     1     0     0     0]
 [    1     0     2   409     2   884     0     0     0]
 [    2     0     1    11     0     0    11     0     0]
 [    0     3    16   127     9     0     0     6     0]
 [    0     0     0    29     

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


# Define the class labels
class_labels = ['N','P', 'A', 'L','R', 'f', 'j','F','a']  # Replace with your actual class labels

# Initialize the confusion matrix
num_classes = len(class_labels)
confusion_matrix = np.zeros((num_classes, num_classes), dtype=int)

# Populate the confusion matrix
for true_label, predicted_label in zip(y_test, poly_pred):
    confusion_matrix[true_label, predicted_label] += 1

# Plot the confusion matrix with labels
plt.imshow(confusion_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()

tick_marks = np.arange(num_classes)
plt.xticks(tick_marks, class_labels)
plt.yticks(tick_marks, class_labels)

thresh = confusion_matrix.max() / 2.0
for i in range(num_classes):
    for j in range(num_classes):
        plt.text(j, i, format(confusion_matrix[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if confusion_matrix[i, j] > thresh else "black")

plt.xlabel('Predicted Label')
plt.ylabel('True Label')

plt.show()


In [ ]:
import joblib

# Assuming you have trained the `poly` kernel SVM model and stored it in the `poly` variable

# Specify the path where you want to save the model
model_path = 'dataset/poly_model.pkl'

# Save the model
joblib.dump(poly, model_path)

['dataset/poly_model.pkl']

In [ ]:
import joblib

# Assuming you have trained the `poly` kernel SVM model and stored it in the `poly` variable

# Specify the path where you want to save the model
model_path = 'dataset/poly_model.pkl'

# Save the model
joblib.dump(poly, model_path)

['dataset/polygan_model.pkl']

In [ ]:
model_path = 'dataset/linear_model.pkl'

# Save the model
joblib.dump(linear, model_path)

model_path = 'dataset/rbf_model.pkl'

# Save the model
joblib.dump(rbf, model_path)

model_path = 'dataset/sig_model.pkl'

# Save the model
joblib.dump(sig, model_path)

['dataset/sig_model.pkl']

NOW GAN TRAIN . Training the model on the generated model and testing on the real data.GAN-train is the accuracy of a classifier trained on Sg and tested on a validation set of real images Sv

In [ ]:
import copy
import csv
import datetime
import json
import os
from collections import Counter

import matplotlib.pyplot as plt
import numpy as np
import torch

import torch.optim as optim

import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset

import sys
#sys.path.append('/content/gdrive/MyDrive/Augmentation-of-ECG-Training-Dataset-with-CGAN-main')

from ekg_class import dicts
from model_ac_wgan_gp_ecg import Disc_ac_wgan_gp_1d, Gen_ac_wgan_gp_1d, initialize_weights_1d
from utils import gradient_penalty, normalize, grid_plot_save, grid_plot

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

num2descr, letter2num, letter2descr, num2letter = dicts()
start_time = datetime.datetime.now()
print(("\n" + "*" * 50 + "\n\t\tstart time:      {0:02d}:{1:02d}:{2:02.0f}\n" + "*" * 50).format(
    start_time.hour, start_time.minute, start_time.second))

#drive = "F:"
drive = ""
myPath_base = os.path.join(drive, "")
myPath_dataset = os.path.join(myPath_base, "")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dry_run = False
if dry_run:
    NUM_EPOCHS = 1
else:
    NUM_EPOCHS = 12
BATCH_SIZE = 16
CHANNELS_IMG = 1
CRITIC_ITERATIONS = 5
FEATURES_DISC = 64
FEATURES_GEN = 64
GEN_EMBEDDING = 100
LAMBDA_GP = 10
LEARNING_RATE = 1e-4
Z_DIM = 100

tr_ts_ratio = 0.1    # "training set / test set" split ratio
len_ratio = 1     # to study the effect of support set number of samples (shorter train sets)
if '.' in str(len_ratio):
    len_ratio_str = str(len_ratio).replace('.', "")
else:
    len_ratio_str = str(len_ratio)

# transforms = transforms.Compose(
#    [
#        transforms.Resize(IMG_SIZE),
#        transforms.ToTensor(),
#        transforms.Normalize(
#            [0.5 for _ in range(CHANNELS_IMG)], [0.5 for _ in range(CHANNELS_IMG)]),
#    ]
# )


with open(os.path.join(myPath_dataset, "gb_together.json"), "r") as f:
    data = json.load(f)

stats_all_classes = Counter(np.asarray(data, dtype=object)[:, 1])

all_classes = ['/', 'A', 'E', 'F', 'J', 'L', 'N', 'Q', 'R', 'S', 'V', 'a', 'e', 'f', 'j']
all_classes_folder = ['P', 'A', 'E', 'F', 'J', 'L', 'N', 'Q', 'R', 'S', 'V', 'a', 'e', 'f', 'j']

all_classes.remove('Q')
all_classes.remove('S')
# all_classes.remove('V')      # why remove this? plots show no-uniform pattern



# %%%%%%%%%%%%%%%%       begin MIT-BIH Dataset      %%%%%%%%%%%%%%%%%%%%%%%%%%%%

d_set = "MIT_BIH"
# Note: '/' and 'P' are the same
classes2keep = ['P','A', 'L', 'R', 'f', 'j','F','a']
classes2keep_folder = ['P','L','R', 'f', 'j','F','a']

# d_set = "ECG5000\\"
# classes2keep = [1, 2, 3, 4, 5]

myPath_save = os.path.join(myPath_base, "PycharmProjects\\paper2_gen_data\\",  "gan_test_report")
os.makedirs(myPath_save, exist_ok=True)

NUM_CLASSES = len(classes2keep)
IMG_SIZE = 256

#with open(os.path.join(myPath_dataset, "record_X_y_adapt_win_bef075_aft075_Normalized.json"), "r") as f:
    #data = json.load(f)

stats_all_classes = Counter(np.asarray(data, dtype=object)[:, 1])

# create dictionary of data to be kept
vals = []
for idx in range(len(classes2keep)):
    vals.append([])

data2keep_dict = dict(zip(classes2keep, vals))
data_train_dict = copy.deepcopy(data2keep_dict)
data_test_dict = copy.deepcopy(data2keep_dict)
count_N=0
count_j=0
for item in data:
    if item[1] in classes2keep:
        data2keep_dict[item[1]].append(item[0])
del data

if '/' in classes2keep:
    temp = data2keep_dict['/']
    data2keep_dict.pop('/')
    data2keep_dict['P'] = temp

for key in classes2keep_folder:
    val_len = len(data2keep_dict[key])
    idx_train = torch.randperm(val_len)[:int(tr_ts_ratio * val_len)]
    idx_test = torch.randperm(val_len)[int(tr_ts_ratio * val_len)+1:]
    data_train_dict[key] = [data2keep_dict[key][idx] for idx in idx_train]
    data_test_dict[key] = [data2keep_dict[key][idx] for idx in idx_test]

X_train_gb = []
y_train_gb = []
# data_train_dict contains all the training data
# X_train contains only a portion of training data (len_ratio), to study of size of training set on
#           classification metrics

a = 0
for key in classes2keep_folder:
    temp = data_train_dict[key]
    length = int(len_ratio*len(temp))
    X_train_gb.extend(temp[:length])
    idx = [classes2keep_folder.index(key)] * length
    # print('key {}, idx {}'.format(key, idx))
    y_train_gb.extend(idx)

X_test_gb = []
y_test_gb = []
for key in classes2keep_folder:
    X_test_gb.extend(data_test_dict[key][:])
    idx = [classes2keep_folder.index(key)] * len(data_test_dict[key])
    y_test_gb.extend(idx)

y_test_stat = Counter(y_test_gb)
y_train_stat = Counter(y_train_gb)

print("train: {}".format(y_train_stat))
print("test: {}".format(y_test_stat))


**************************************************
		start time:      16:15:51
**************************************************
train: Counter({0: 1001, 2: 1001, 6: 1001, 1: 1000, 3: 1000, 4: 1000, 7: 1000, 5: 900})
test: Counter({0: 9010, 2: 9008, 6: 9008, 3: 9007, 4: 9004, 1: 9001, 7: 8999, 5: 8100})


In [ ]:
# Initialize feature vector and labels
feature_vector = []
labels = []

# Iterate over the beats in the dictionary
for cls, beats in data_test_dict.items():
    for beat in beats:
        # Perform wavelet decomposition at 4 levels
        coeffs = pywt.wavedec(beat, 'db6', level=4)

        # Extract approximation coefficients at level 4
        approx_coeffs = coeffs[0]

        # Append approximation coefficients to feature vector
        feature_vector.append(approx_coeffs)

        # Append label for the beat
        labels.append(cls)  # Assuming the record number is the label

# Convert feature vector and labels to numpy arrays
feature_vector = np.array(feature_vector)
labels = np.array(labels)

In [ ]:
X_train_gb, X_test_gb, y_train_gb, y_test_gb = train_test_split(feature_vector, labels, train_size=0.8, random_state = 0)

In [ ]:
#linear = svm.SVC(kernel='linear', C=1, decision_function_shape='ovo').fit(X_train, y_train)
#rbf = svm.SVC(kernel='rbf', gamma=1, C=1, decision_function_shape='ovo').fit(X_train, y_train)
poly = svm.SVC(kernel='poly', degree=3, C=1, decision_function_shape='ovo').fit(X_train_gb, y_train_gb)
#sig = svm.SVC(kernel='sigmoid', C=1, decision_function_shape='ovo').fit(X_train, y_train)

In [ ]:
#linear_pred = linear.predict(X_test_gb)
poly_pred = poly.predict(X_test_gb)
#rbf_pred = rbf.predict(X_test_gb)
#sig_pred = sig.predict(X_test_gb)

In [ ]:
#accuracy_lin = linear.score(X_test_gb, y_test_gb)
accuracy_poly = poly.score(X_test_gb, y_test_gb)
#accuracy_rbf = rbf.score(X_test_gb, y_test_gb)
#accuracy_sig = sig.score(X_test_gb, y_test_gb)
#print("Accuracy Linear Kernel:", accuracy_lin)
print("Accuracy Polynomial Kernel:", accuracy_poly)
#print("Accuracy Radial Basis Kernel:", accuracy_rbf)
#print("Accuracy Sigmoid Kernel:", accuracy_sig)

Accuracy Polynomial Kernel: 0.9999297160528535


In [ ]:
# creating a confusion matrix
#cm_lin = confusion_matrix(y_test_gb, linear_pred)
cm_poly = confusion_matrix(y_test_gb, poly_pred)
#cm_rbf = confusion_matrix(y_test_gb, rbf_pred)
#cm_sig = confusion_matrix(y_test_gb, sig_pred)
#print(cm_lin)
print(cm_poly)
#print(cm_rbf)
#print(cm_sig)

[[1832    0    0    0    0    0    0    0]
 [   0 1757    0    0    1    0    0    0]
 [   0    0 1757    0    0    0    0    0]
 [   0    0    0 1775    0    0    0    0]
 [   0    0    0    0 1819    0    0    0]
 [   0    0    0    0    0 1779    0    0]
 [   0    0    0    0    0    0 1868    0]
 [   0    0    0    0    0    0    0 1640]]


In [ ]:
#creating test dataset from real data( )

#making the test dataset from real data

import copy
import csv
import datetime
import json
import os
from collections import Counter

import matplotlib.pyplot as plt
import numpy as np
import torch

import torch.optim as optim

import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset

import sys
#sys.path.append('/content/gdrive/MyDrive/Augmentation-of-ECG-Training-Dataset-with-CGAN-main')

from ekg_class import dicts
from model_ac_wgan_gp_ecg import Disc_ac_wgan_gp_1d, Gen_ac_wgan_gp_1d, initialize_weights_1d
from utils import gradient_penalty, normalize, grid_plot_save, grid_plot

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

num2descr, letter2num, letter2descr, num2letter = dicts()
start_time = datetime.datetime.now()
print(("\n" + "*" * 50 + "\n\t\tstart time:      {0:02d}:{1:02d}:{2:02.0f}\n" + "*" * 50).format(
    start_time.hour, start_time.minute, start_time.second))

#drive = "F:"
drive = ""
myPath_base = os.path.join(drive, "")
myPath_dataset = os.path.join(myPath_base, "")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dry_run = False
if dry_run:
    NUM_EPOCHS = 1
else:
    NUM_EPOCHS = 12
BATCH_SIZE = 16
CHANNELS_IMG = 1
CRITIC_ITERATIONS = 5
FEATURES_DISC = 64
FEATURES_GEN = 64
GEN_EMBEDDING = 100
LAMBDA_GP = 10
LEARNING_RATE = 1e-4
Z_DIM = 100

tr_ts_ratio = 0.1     # "training set / test set" split ratio
len_ratio = 1     # to study the effect of support set number of samples (shorter train sets)
if '.' in str(len_ratio):
    len_ratio_str = str(len_ratio).replace('.', "")
else:
    len_ratio_str = str(len_ratio)

# transforms = transforms.Compose(
#    [
#        transforms.Resize(IMG_SIZE),
#        transforms.ToTensor(),
#        transforms.Normalize(
#            [0.5 for _ in range(CHANNELS_IMG)], [0.5 for _ in range(CHANNELS_IMG)]),
#    ]
# )


with open(os.path.join(myPath_dataset, "record_X_y_adapt_win_bef075_aft075_Normalized.json"), "r") as f:
    data = json.load(f)

stats_all_classes = Counter(np.asarray(data, dtype=object)[:, 2])

all_classes = ['/', 'A', 'E', 'F', 'J', 'L', 'N', 'Q', 'R', 'S', 'V', 'a', 'e', 'f', 'j']
all_classes_folder = ['P', 'A', 'E', 'F', 'J', 'L', 'N', 'Q', 'R', 'S', 'V', 'a', 'e', 'f', 'j']

all_classes.remove('Q')
all_classes.remove('S')
# all_classes.remove('V')      # why remove this? plots show no-uniform pattern

# %%%%%%%%%%%%%%%%%    begin classes statistics      %%%%%%%%%%%%%%%%%%
"""
stats_all_classes =
        {
        'N': 75052,   'L': 8075,  'R': 7259,  'V': 7130,
        '/': 7028,    'A': 2546,  'f': 982,   'F': 803,
        'j': 229,     'a': 150,   'E': 106,   'J': 83,
        'Q': 33,      'e': 16,    'S': 2
        }
P: Paced beat
A: Atrial Premature contraction
E: Ventricular Escape beat
F: Fusion of Ventricular and Normal beat
J: Nodal (junctional) Premature Beat
L: Left bundle branch block beat
N: Normal beat
Q: Unclassifiable beat
R: Right bundle branch block beat
S: Premature or ectopic supraventricular beat
V: Premature Ventricular Contraction
a: Aberrated Atrial Premature beat
e: Atrial escape beat
f: Fusion of paced and normal beat
j: Nodal (junctional) escape beat
"""
# %%%%%%%%%%%%%%%%%    end classes statistics    %%%%%%%%%%%%%%%%%%%

# %%%%%%%%%%%%%%%%       begin MIT-BIH Dataset      %%%%%%%%%%%%%%%%%%%%%%%%%%%%

d_set = "MIT_BIH"
# Note: '/' and 'P' are the same
#classes2keep = ['/', 'A', 'L', 'N', 'R', 'f', 'j']
#classes2keep_folder = ['P', 'A', 'L', 'N', 'R', 'f', 'j']
classes2keep = [ 'L', 'R', 'f', 'j','F','a']
classes2keep_folder = ['L','R', 'f', 'j','F','a']

# d_set = "ECG5000\\"
# classes2keep = [1, 2, 3, 4, 5]

myPath_save = os.path.join(myPath_base, "PycharmProjects\\paper2_gen_data\\", d_set, "multiclass",
                            "genbeats_ac_wgan_gp_cl_{}_len_ratio_{}".format(classes2keep_folder, len_ratio_str))
os.makedirs(myPath_save, exist_ok=True)

NUM_CLASSES = len(classes2keep)
IMG_SIZE = 256

#with open(os.path.join(myPath_dataset, "record_X_y_adapt_win_bef075_aft075_Normalized.json"), "r") as f:
    #data = json.load(f)

stats_all_classes = Counter(np.asarray(data, dtype=object)[:, 2])

# create dictionary of data to be kept
vals = []
for idx in range(len(classes2keep)):
    vals.append([])

data2keep_dict = dict(zip(classes2keep, vals))
data_train_dict = copy.deepcopy(data2keep_dict)
data_test_dict = copy.deepcopy(data2keep_dict)

for item in data:
    if item[2] in classes2keep:
        data2keep_dict[item[2]].append(item[1])
del data

if '/' in classes2keep:
    temp = data2keep_dict['/']
    data2keep_dict.pop('/')
    data2keep_dict['P'] = temp


max_items_per_class = 1400



with open(os.path.join(myPath_dataset, "gb_together.json"), "r") as f:
    data_gb = json.load(f)
classes2gb=['L','R','f']

class_counts = {class_label: 0 for class_label in classes2gb}


for item in data_gb:
    if item[1] in classes2gb and class_counts[item[1]] < max_items_per_class:
        data2keep_dict[item[1]].append(item[0])
        class_counts[item[1]] += 1

del data_gb


# randomly splitting the dataset into train and test sets
for key in classes2keep_folder:
    val_len = len(data2keep_dict[key])
    idx_train = torch.randperm(val_len)[:int(tr_ts_ratio * val_len)]
    idx_test = torch.randperm(val_len)[int(tr_ts_ratio * val_len)+1:]
    data_train_dict[key] = [data2keep_dict[key][idx] for idx in idx_train]
    data_test_dict[key] = [data2keep_dict[key][idx] for idx in idx_test]

X_train = []
y_train = []
# data_train_dict contains all the training data
# X_train contains only a portion of training data (len_ratio), to study of size of training set on
#           classification metrics

a = 0
for key in classes2keep_folder:
    temp = data_train_dict[key]
    length = int(len_ratio*len(temp))
    X_train.extend(temp[:length])
    idx = [classes2keep_folder.index(key)] * length
    # print('key {}, idx {}'.format(key, idx))
    y_train.extend(idx)

X_test = []
y_test = []
for key in classes2keep_folder:
    X_test.extend(data_test_dict[key][:])
    idx = [classes2keep_folder.index(key)] * len(data_test_dict[key])
    y_test.extend(idx)

y_test_stat = Counter(y_test)
y_train_stat = Counter(y_train)

print("train: {}".format(y_train_stat))
print("test: {}".format(y_test_stat))


**************************************************
		start time:      17:16:37
**************************************************
train: Counter({0: 7505, 1: 807, 2: 725, 3: 98, 5: 80, 4: 22, 6: 15})
test: Counter({0: 67546, 1: 7267, 2: 6533, 3: 883, 5: 722, 4: 206, 6: 134})


In [ ]:
# Initialize feature vector and labels
feature_vector = []
labels = []

# Iterate over the beats in the dictionary
for cls, beats in data_test_dict.items():
    for beat in beats:
        # Perform wavelet decomposition at 4 levels
        coeffs = pywt.wavedec(beat, 'db6', level=4)

        # Extract approximation coefficients at level 4
        approx_coeffs = coeffs[0]

        # Append approximation coefficients to feature vector
        feature_vector.append(approx_coeffs)

        # Append label for the beat
        labels.append(cls)  # Assuming the record number is the label

# Convert feature vector and labels to numpy arrays
feature_vector = np.array(feature_vector)
labels = np.array(labels)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(feature_vector, labels, train_size=0.9, random_state = 0)

In [ ]:
y_test_stat = Counter(y_test)
y_train_stat = Counter(y_train)
print("train: {}".format(y_train_stat))
print("test: {}".format(y_test_stat))

train: Counter({'L': 7669, 'R': 7045, 'f': 1914, 'F': 650, 'j': 185, 'a': 109})
test: Counter({'L': 858, 'R': 748, 'f': 229, 'F': 72, 'a': 25, 'j': 21})


In [ ]:
accuracy_poly = poly.score(X_test, y_test)
#accuracy_rbf = rbf.score(X_test_gb, y_test_gb)
#accuracy_sig = sig.score(X_test_gb, y_test_gb)
#print("Accuracy Linear Kernel:", accuracy_lin)
print("Accuracy Polynomial Kernel:", accuracy_poly)

Accuracy Polynomial Kernel: 0.7014848950332822


In [ ]:
poly_pred = poly.predict(X_test)
cm_poly = confusion_matrix(y_test, poly_pred)
#cm_rbf = confusion_matrix(y_test_gb, rbf_pred)
#cm_sig = confusion_matrix(y_test_gb, sig_pred)
#print(cm_lin)
print(cm_poly)

[[ 507   90   16   26   37    0   15   20]
 [   2 5697   14 1359  118    0   72  208]
 [   0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0]
 [ 345  391  242 1435 3749    0   75  499]
 [   5   28   18    0   35   20   17   10]
 [ 125  454   25   47   91    0  387    7]
 [   1   39   51   21   10    0   18   64]]


Now gan-test . ,For getting the GAN-test score, SVM needs to be trained on the original ECG sample and tested on the generated ECG sample. The higher the GAN-test score, the better the fidelity of the generated data . Here we are keeping the test size same as the real dataset that was used for training the template model.

In [ ]:
import joblib

# Specify the path where the model is saved
model_path = 'dataset/poly_model.pkl'

# Load the saved model
poly= joblib.load(model_path)

model_path = 'dataset/linear_model.pkl'

# Load the saved model
linear= joblib.load(model_path)

model_path = 'dataset/rbf_model.pkl'

# Load the saved model
rbf= joblib.load(model_path)

model_path = 'dataset/sig_model.pkl'

# Load the saved model
sig= joblib.load(model_path)

In [ ]:
import copy
import csv
import datetime
import json
import os
from collections import Counter

import matplotlib.pyplot as plt
import numpy as np
import torch

import torch.optim as optim

import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset

import sys
#sys.path.append('/content/gdrive/MyDrive/Augmentation-of-ECG-Training-Dataset-with-CGAN-main')

from ekg_class import dicts
from model_ac_wgan_gp_ecg import Disc_ac_wgan_gp_1d, Gen_ac_wgan_gp_1d, initialize_weights_1d
from utils import gradient_penalty, normalize, grid_plot_save, grid_plot

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

num2descr, letter2num, letter2descr, num2letter = dicts()
start_time = datetime.datetime.now()
print(("\n" + "*" * 50 + "\n\t\tstart time:      {0:02d}:{1:02d}:{2:02.0f}\n" + "*" * 50).format(
    start_time.hour, start_time.minute, start_time.second))

#drive = "F:"
drive = ""
myPath_base = os.path.join(drive, "")
myPath_dataset = os.path.join(myPath_base, "")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dry_run = False
if dry_run:
    NUM_EPOCHS = 1
else:
    NUM_EPOCHS = 12
BATCH_SIZE = 16
CHANNELS_IMG = 1
CRITIC_ITERATIONS = 5
FEATURES_DISC = 64
FEATURES_GEN = 64
GEN_EMBEDDING = 100
LAMBDA_GP = 10
LEARNING_RATE = 1e-4
Z_DIM = 100

tr_ts_ratio = 0.0    # "training set / test set" split ratio
len_ratio = 1     # to study the effect of support set number of samples (shorter train sets)
if '.' in str(len_ratio):
    len_ratio_str = str(len_ratio).replace('.', "")
else:
    len_ratio_str = str(len_ratio)

# transforms = transforms.Compose(
#    [
#        transforms.Resize(IMG_SIZE),
#        transforms.ToTensor(),
#        transforms.Normalize(
#            [0.5 for _ in range(CHANNELS_IMG)], [0.5 for _ in range(CHANNELS_IMG)]),
#    ]
# )


with open(os.path.join(myPath_dataset, "gb_together.json"), "r") as f:
    data = json.load(f)

stats_all_classes = Counter(np.asarray(data, dtype=object)[:, 1])

all_classes = ['/', 'A', 'E', 'F', 'J', 'L', 'N', 'Q', 'R', 'S', 'V', 'a', 'e', 'f', 'j']
all_classes_folder = ['P', 'A', 'E', 'F', 'J', 'L', 'N', 'Q', 'R', 'S', 'V', 'a', 'e', 'f', 'j']

all_classes.remove('Q')
all_classes.remove('S')
# all_classes.remove('V')      # why remove this? plots show no-uniform pattern



# %%%%%%%%%%%%%%%%       begin MIT-BIH Dataset      %%%%%%%%%%%%%%%%%%%%%%%%%%%%

d_set = "MIT_BIH"
# Note: '/' and 'P' are the same
classes2keep = ['N','P','A' ,'L', 'R', 'f', 'j','F','a']
classes2keep_folder = ['N','P','A','L','R', 'f', 'j','F','a']

# d_set = "ECG5000\\"
# classes2keep = [1, 2, 3, 4, 5]

myPath_save = os.path.join(myPath_base, "PycharmProjects\\paper2_gen_data\\",  "gan_test_report")
os.makedirs(myPath_save, exist_ok=True)

NUM_CLASSES = len(classes2keep)
IMG_SIZE = 256

#with open(os.path.join(myPath_dataset, "record_X_y_adapt_win_bef075_aft075_Normalized.json"), "r") as f:
    #data = json.load(f)

stats_all_classes = Counter(np.asarray(data, dtype=object)[:, 1])

# create dictionary of data to be kept
vals = []
for idx in range(len(classes2keep)):
    vals.append([])

data2keep_dict = dict(zip(classes2keep, vals))
data_train_dict = copy.deepcopy(data2keep_dict)
data_test_dict = copy.deepcopy(data2keep_dict)
count_f=0
count_j=0
count_F=0
count_a=0
count_P=0
count_A=0
count_L=0
count_R=0
count_N=0
for item in data:
    if item[1] == 'f'  and count_f< 99:  # Add condition to limit 'N' data to 4000
        data2keep_dict[item[1]].append(item[0])
        count_f += 1
    if item[1] == 'j'  and count_j < 23:  # Add condition to limit 'N' data to 4000
        data2keep_dict[item[1]].append(item[0])
        count_j += 1
    if item[1] == 'F'  and count_F < 81:  # Add condition to limit 'N' data to 4000
        data2keep_dict[item[1]].append(item[0])
        count_F += 1

    if item[1] == 'a'  and count_a < 15:  # Add condition to limit 'N' data to 4000
        data2keep_dict[item[1]].append(item[0])
        count_a += 1

    if item[1] == 'A'  and count_A< 255:  # Add condition to limit 'N' data to 4000
        data2keep_dict[item[1]].append(item[0])
        count_A += 1

    if item[1] == 'P'  and count_P< 702:  # Add condition to limit 'N' data to 4000
        data2keep_dict[item[1]].append(item[0])
        count_P += 1
    if item[1] == 'R'  and count_R< 726:  # Add condition to limit 'N' data to 4000
        data2keep_dict[item[1]].append(item[0])
        count_R += 1
    if item[1] == 'N'  and count_N< 7506:  # Add condition to limit 'N' data to 4000
        data2keep_dict[item[1]].append(item[0])
        count_N += 1
    elif item[1] == 'L'  and count_L< 808:  # Add condition to limit 'N' data to 4000
        data2keep_dict[item[1]].append(item[0])
        count_L += 1

del data

if '/' in classes2keep:
    temp = data2keep_dict['/']
    data2keep_dict.pop('/')
    data2keep_dict['P'] = temp

for key in classes2keep_folder:
    val_len = len(data2keep_dict[key])
    idx_train = torch.randperm(val_len)[:int(tr_ts_ratio * val_len)]
    idx_test = torch.randperm(val_len)[int(tr_ts_ratio * val_len)+1:]
    data_train_dict[key] = [data2keep_dict[key][idx] for idx in idx_train]
    data_test_dict[key] = [data2keep_dict[key][idx] for idx in idx_test]

X_train_gb = []
y_train_gb = []
# data_train_dict contains all the training data
# X_train contains only a portion of training data (len_ratio), to study of size of training set on
#           classification metrics

a = 0
for key in classes2keep_folder:
    temp = data_train_dict[key]
    length = int(len_ratio*len(temp))
    X_train_gb.extend(temp[:length])
    idx = [classes2keep_folder.index(key)] * length
    # print('key {}, idx {}'.format(key, idx))
    y_train_gb.extend(idx)

X_test_gb = []
y_test_gb = []
for key in classes2keep_folder:
    X_test_gb.extend(data_test_dict[key][:])
    idx = [classes2keep_folder.index(key)] * len(data_test_dict[key])
    y_test_gb.extend(idx)

y_test_stat = Counter(y_test_gb)
y_train_stat = Counter(y_train_gb)

print("train: {}".format(y_train_stat))
print("test: {}".format(y_test_stat))


**************************************************
		start time:      17:03:36
**************************************************
train: Counter()
test: Counter({0: 7505, 3: 807, 4: 725, 1: 701, 2: 254, 5: 98, 7: 80, 6: 22, 8: 14})


In [ ]:
# Initialize feature vector and labels
feature_vector = []
labels = []

# Iterate over the beats in the dictionary
for cls, beats in data_test_dict.items():
    for beat in beats:
        # Perform wavelet decomposition at 4 levels
        coeffs = pywt.wavedec(beat, 'db6', level=4)

        # Extract approximation coefficients at level 4
        approx_coeffs = coeffs[0]

        # Append approximation coefficients to feature vector
        feature_vector.append(approx_coeffs)

        # Append label for the beat
        labels.append(cls)  # Assuming the record number is the label

# Convert feature vector and labels to numpy arrays
feature_vector = np.array(feature_vector)
labels = np.array(labels)

X_train_gb, X_test_gb, y_train_gb, y_test_gb = train_test_split(feature_vector, labels, train_size=1, random_state = 0)

In [ ]:
y_test_stat = Counter(y_test_gb)
y_train_stat = Counter(y_train_gb)

print("train: {}".format(y_train_stat))
print("test: {}".format(y_test_stat))


poly_pred = poly.predict(X_test_gb)


accuracy_poly = poly.score(X_test_gb, y_test_gb)

print("Accuracy Polynomial Kernel:", accuracy_poly)

train: Counter({'N': 1})
test: Counter({'N': 7504, 'L': 807, 'R': 725, 'P': 701, 'A': 254, 'f': 98, 'F': 80, 'j': 22, 'a': 14})
Accuracy Polynomial Kernel: 0.9304262616364527


In [ ]:
cm_poly = confusion_matrix(y_test_gb, poly_pred)
#cm_rbf = confusion_matrix(y_test_gb, rbf_pred)
#cm_sig = confusion_matrix(y_test_gb, sig_pred)
#print(cm_lin)
print(cm_poly)

[[  19    0    0  235    0    0    0    0    0]
 [   0   78    0    2    0    0    0    0    0]
 [   0    1  750   55    1    0    0    0    0]
 [   0    0    1 7503    0    0    0    0    0]
 [   0    0    0    8  693    0    0    0    0]
 [   0    0    0  275    0  450    0    0    0]
 [   0    0    0   12    0    0    2    0    0]
 [   0    0    2   96    0    0    0    0    0]
 [   0    0    0   22    0    0    0    0    0]]
